# Scott Breitbach
## Milestone 5: Merging the Data and Storing in a Database/Visualizing Data
## 26-Feb-2021
## DSC540, Weeks 11-12

In [1]:
import sqlite3
import pandas as pd

## Load data into an SQL database, each as an individual table.

In [2]:
csvData = pd.read_csv("CSV_nebraskaCleaned.csv")
webData = pd.read_csv("WEB_NEbeerReddit.csv")
apiData = pd.read_csv("API_beerMapping.csv")

## Join the datasets together into 1 dataset.

In [3]:
with sqlite3.connect('Nebeerska.db') as conn:
    cursor = conn.cursor()
    csvData.to_sql('csvtable', conn, if_exists='replace', index=False)
    webData.to_sql('webtable', conn, if_exists='replace', index=False)
    apiData.to_sql('apitable', conn, if_exists='replace', index=False)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


### Get a list of the tables:

In [4]:
res = conn.execute('''SELECT name FROM sqlite_master WHERE type='table';''')

for name in res:
    print(name[0])

csvtable
webtable
apitable


### Check out the first few rows of each table:

In [12]:
def head(table, nRows=5):
    cursor.execute(f'''SELECT * FROM {table}''')
    for row in cursor.fetchall()[:nRows]:
        print(row)

In [14]:
head('csvtable', 3)

('backswing-brewing-co-lincoln', 'Backswing Brewing Co.', 'Micro', '500 W South St Ste 8', 'Lincoln', 'Nebraska', '68522-1744', 68522, 'http://www.backswingbrewing.com', '(402) 515-4263', None, None)
('benson-brewery-omaha', 'Benson Brewery', 'Brewpub', '6059 Maple St', 'Omaha', 'Nebraska', '68104-4050', 68104, 'http://www.bensonbrewery.com', '(402) 934-8668', -96.00634511, 41.28489756)
('blue-blood-brewing-company-lincoln', 'Blue Blood Brewing Company', 'Brewpub', '925 Robbers Cave Rd', 'Lincoln', 'Nebraska', '68502', 68502, 'http://www.bluebloodbrewing.com', '(402) 477-2337', -96.70701376, 40.77999602)


In [15]:
head('webtable', 3)

('Backswing Brewing Co.', 'Lincoln', 'Nebraska', 'Micro', 'On-tap at the brewery and at select locations', 'Distro planned for 2016', 1, 'Lincoln')
('Benson Brewery', 'Omaha', 'Nebraska', 'Taproom', 'On tap only at the brewery', 'Other beers, locals included, on tap. Omaha Brewing is the parent company', 1, 'Omaha')
('Boiler Brewing Company', 'Lincoln', 'Nebraska', 'Taproom', 'On tap only at the brewery', 'Tours posted on FB page', 2, 'Lincoln / Lincoln - South')


In [16]:
head('apitable', 3)

(21938, 'Catalyst Brewing Company', 'Brewery', 'https://beermapping.com/location/21938', 'http://beermapping.com/maps/proxymaps.php?locid=21938&amp;d=5', 'http://beermapping.com/maps/blogproxy.php?locid=21938&amp;d=1&amp;type=norm', '5730 Hidcote Drive', 'Lincoln', 'NE', 68516, 'United States', '402-875-5588', None, 0.0, 0)
(19644, 'Backswing Brewing Co.', 'Brewery', 'https://beermapping.com/location/19644', 'http://beermapping.com/maps/proxymaps.php?locid=19644&amp;d=5', 'http://beermapping.com/maps/blogproxy.php?locid=19644&amp;d=1&amp;type=norm', '500 W South St #8', 'Lincoln', 'NE', 68522, 'United States', '(402) 515-4263', 'facebook.com/BackswingBrewingCo/timeline', 0.0, 0)
(17706, 'Benson Brewery', 'Brewpub', 'https://beermapping.com/location/17706', 'http://beermapping.com/maps/proxymaps.php?locid=17706&amp;d=5', 'http://beermapping.com/maps/blogproxy.php?locid=17706&amp;d=1&amp;type=norm', '6059 Maple Street', 'Omaha', 'NE', 68104, 'United States', '(402) 937-1892', 'bensonbrew

## Create 5 visualizations that demonstrate the data.
Note: At least 2 should have data from > 1 source.